# Imports

In [1]:
import json
import time

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from pydantic import BaseModel
from typing import List, Optional

from selenium import webdriver

from supabase import create_client, Client

# Create scraping pipeline

## Driver

In [2]:
driver = webdriver.Chrome()
#page url
whoscored_url = "https://www.whoscored.com/Matches/1734706/Live/Spain-LaLiga-2023-2024-Rayo-Vallecano-Barcelona"
#driver returns page
driver.get(whoscored_url)

## Find out the information through out the page code

In [3]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
#We put script because it contains our data target
element = soup.select_one('script:-soup-contains("matchCentreData")')

In [5]:
matchdict = json.loads(element.text.split("matchCentreData: ")[1].split(',\n')[0])

In [6]:
matchdict.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

## Create a dataframe with the match event data

In [7]:
match_events = matchdict['events']
df = pd.DataFrame(match_events).dropna(subset='playerId')

## Clean the dataframe 

In [8]:
df = df.where(pd.notnull(df), None)

In [9]:
def convert_to_snake_case(column_name):
    result = [column_name[0].lower()]
    for char in column_name[1:]:
        if char.isupper():
            result.extend(['_', char.lower()])
        else:
            result.append(char)
    return ''.join(result)

# Rename columns
df.columns = [convert_to_snake_case(col) for col in df.columns]

In [10]:
df['period_display_name'] = df['period'].apply(lambda x: x['displayName'])
df['type_display_name'] = df['type'].apply(lambda x: x['displayName'])
df['outcome_type_display_name'] = df['outcome_type'].apply(lambda x: x['displayName'])

df.drop(columns=["period","type","outcome_type"], inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1521 entries, 2 to 1526
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         1521 non-null   float64
 1   event_id                   1521 non-null   int64  
 2   minute                     1521 non-null   int64  
 3   second                     1517 non-null   float64
 4   team_id                    1521 non-null   int64  
 5   x                          1521 non-null   float64
 6   y                          1521 non-null   float64
 7   expanded_minute            1521 non-null   int64  
 8   qualifiers                 1521 non-null   object 
 9   satisfied_events_types     1521 non-null   object 
 10  is_touch                   1521 non-null   bool   
 11  player_id                  1521 non-null   float64
 12  end_x                      988 non-null    float64
 13  end_y                      988 non-null    float

In [12]:
df = df[[
    'id', 'event_id', 'minute', 'second', 'team_id', 'player_id', 'x', 'y', 'end_x', 'end_y',
    'qualifiers', 'is_touch', 'blocked_x', 'blocked_y', 'goal_mouth_z', 'goal_mouth_y', 'is_shot',
    'card_type', 'is_goal', 'type_display_name', 'outcome_type_display_name',
    'period_display_name'
]]

In [13]:
df[['id', 'event_id', 'minute', 'team_id', 'player_id']] = df[['id', 'event_id', 'minute', 'team_id', 'player_id']].astype(np.int64)
df[['second', 'x', 'y', 'end_x', 'end_y']] = df[['second', 'x', 'y', 'end_x', 'end_y']].astype(float)
df[['is_shot', 'is_goal', 'card_type']] = df[['is_shot', 'is_goal', 'card_type']].astype(bool)

In [14]:
df['is_goal'] = df['is_goal'].fillna(False)
df['is_shot'] = df['is_shot'].fillna(False)

In [15]:
for column in df.columns:
    if df[column].dtype == np.float64 or df[column].dtype == np.float32:
        df[column] = np.where(
            np.isnan(df[column]),
            None,
            df[column]
        )

### Generate match id

In [16]:
import re
timestamp = re.split('[-: ]', matchdict['timeStamp'])
teams = [str(i) for i in df.team_id.unique()]
match_id = int(''.join(timestamp + teams))

In [17]:
match_id

202311260939156564

In [18]:
df['match_id']=match_id

### Create player_name column

In [20]:
df_names = pd.DataFrame(list(matchdict['playerIdNameDictionary'].items()), columns=['PlayerId', 'PlayerName'])
df_names['PlayerId']=df_names['PlayerId'].astype(np.int64)
df_names.head()

,PlayerId,PlayerName
0,101613,Iván Balliu
1,16942,Óscar Trejo
2,18701,Radamel Falcao
3,80758,Florian Lejeune
4,400844,Abdul Mumin


In [21]:
df = df.merge(df_names, left_on='player_id', right_on='PlayerId', how ='left').drop('PlayerId', axis=1)

In [22]:
df.tail()

,id,event_id,minute,second,team_id,player_id,x,y,end_x,end_y,...,goal_mouth_z,goal_mouth_y,is_shot,card_type,is_goal,type_display_name,outcome_type_display_name,period_display_name,match_id,PlayerName
1516,2620446911,1018,98,24.0,65,106885,12.4,41.0,None,None,...,None,None,False,False,False,OffsideProvoked,Successful,SecondHalf,202311260939156564,Íñigo Martínez
1517,2620447097,1019,98,42.0,65,367164,18.5,49.1,25.9,63.3,...,None,None,False,False,False,Pass,Successful,SecondHalf,202311260939156564,Iñaki Peña
1518,2620447185,1020,98,45.0,65,106885,28.3,62.4,83.9,0.0,...,None,None,False,False,False,Pass,Unsuccessful,SecondHalf,202311260939156564,Íñigo Martínez
1519,2620447329,692,98,57.0,64,143655,18.5,100.0,4.7,82.9,...,None,None,False,False,False,Pass,Successful,SecondHalf,202311260939156564,Alfonso Espino
1520,2620447435,693,98,58.0,64,80758,4.4,82.1,55.6,32.4,...,None,None,False,False,False,Pass,Unsuccessful,SecondHalf,202311260939156564,Florian Lejeune


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1521 entries, 0 to 1520
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         1521 non-null   int64 
 1   event_id                   1521 non-null   int64 
 2   minute                     1521 non-null   int64 
 3   second                     1517 non-null   object
 4   team_id                    1521 non-null   int64 
 5   player_id                  1521 non-null   int64 
 6   x                          1521 non-null   object
 7   y                          1521 non-null   object
 8   end_x                      988 non-null    object
 9   end_y                      988 non-null    object
 10  qualifiers                 1521 non-null   object
 11  is_touch                   1521 non-null   bool  
 12  blocked_x                  16 non-null     object
 13  blocked_y                  16 non-null     object
 14  goal_mou

In [24]:
class MatchEvent(BaseModel):
    id: int
    event_id: int
    minute: int
    second: Optional[float] = None
    team_id: int
    player_id: int
    x: float
    y: float
    end_x: Optional[float] = None
    end_y: Optional[float] = None
    qualifiers: List[dict]
    is_touch: bool
    blocked_x: Optional[float] = None
    blocked_y: Optional[float] = None
    goal_mouth_z: Optional[float] = None
    goal_mouth_y: Optional[float] = None
    is_shot: bool
    card_type: bool
    is_goal: bool
    type_display_name: str
    outcome_type_display_name: str
    period_display_name: str

In [25]:
for x in df.to_dict(orient="records"):
    try:
        MatchEvent(**x).model_dump()
    except Exception as e:
        print(e)
        break

In [26]:
x

{'id': 2620447435,
 'event_id': 693,
 'minute': 98,
 'second': 58.0,
 'team_id': 64,
 'player_id': 80758,
 'x': 4.4,
 'y': 82.1,
 'end_x': 55.6,
 'end_y': 32.4,
 'qualifiers': [{'type': {'value': 56, 'displayName': 'Zone'},
   'value': 'Center'},
  {'type': {'value': 212, 'displayName': 'Length'}, 'value': '63.5'},
  {'type': {'value': 141, 'displayName': 'PassEndY'}, 'value': '32.4'},
  {'type': {'value': 178, 'displayName': 'StandingSave'}},
  {'type': {'value': 140, 'displayName': 'PassEndX'}, 'value': '55.6'},
  {'type': {'value': 1, 'displayName': 'Longball'}},
  {'type': {'value': 213, 'displayName': 'Angle'}, 'value': '5.72'}],
 'is_touch': True,
 'blocked_x': None,
 'blocked_y': None,
 'goal_mouth_z': None,
 'goal_mouth_y': None,
 'is_shot': False,
 'card_type': False,
 'is_goal': False,
 'type_display_name': 'Pass',
 'outcome_type_display_name': 'Unsuccessful',
 'period_display_name': 'SecondHalf',
 'match_id': 202311260939156564,
 'PlayerName': 'Florian Lejeune'}

In [27]:
supabase_password = 'password'

In [28]:
project_url = 'your-project-url'
api_key = 'your-api-key'

In [29]:
def insert_match_events(df, supabase):
    events = [
        MatchEvent(**x).model_dump()
        for x in df.to_dict(orient='records')
    ]
    
    execution = supabase.table('match_events').upsert(events).execute()

In [ ]:
supabase = create_client(project_url, api_key)

In [ ]:
insert_match_events(df, supabase)

## Functions to automate it

In [31]:
#Imports
import json
import time

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from pydantic import BaseModel
from typing import List, Optional, Dict, Any

from selenium import webdriver

from supabase import create_client, Client

#Functions

def convert_to_snake_case(column_name: str) -> str:
    """
    Convert a column name to snake_case.

    Parameters:
    - column_name (str): The column name to be converted.

    Returns:
    str: The converted column name in snake_case.
    """
    result = [column_name[0].lower()]
    for char in column_name[1:]:
        result.extend(['_', char.lower()] if char.isupper() else [char])
    return ''.join(result)

def get_matchdata_keys(url: str) -> Dict[str, Any]:
    """
    Retrieve match data from a Whoscored URL.

    Parameters:
    - url (str): The Whoscored URL for the desired match.

    Returns:
    Tuple[Dict[str, Any], KeysView[str]]: A tuple containing the match data dictionary
    and a view of its keys.
    """

    # Use a context manager to ensure the webdriver is closed properly
    with webdriver.Chrome() as driver:
        driver.get(url)
        # Use BeautifulSoup to parse the page content
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        # Locate the script containing matchCentreData
        element = soup.select_one('script:-soup-contains("matchCentreData")')
        # Extract and parse the relevant JSON data
        matchdict = json.loads(element.text.split("matchCentreData: ")[1].split(',\n')[0])
        #Get matchdict keys
        matchdict_keys = matchdict.keys()

    return matchdict,matchdict_keys

def get_data(url: str, key: str) -> pd.DataFrame:
    """
    Extract and preprocess match data based on the specified key.

    Parameters:
    - url (str): The Whoscored URL for the desired match.
    - key (str): The key specifying the type of data to extract.

    Returns:
    pd.DataFrame: A DataFrame containing the extracted and processed match data.
    """
    match_data, match_keys = get_matchdata_keys(url)
    
    if key == 'events':
        df = pd.DataFrame(match_data[key])
        df = df.dropna(subset='playerId')
        df = df.where(pd.notnull(df), None)
        
        # Convert column names to snake_case
        df.columns = [convert_to_snake_case(col) for col in df.columns]
        
        # Extract additional information from nested dictionaries
        df['period_display_name'] = df['period'].apply(lambda x: x['displayName'])
        df['type_display_name'] = df['type'].apply(lambda x: x['displayName'])
        df['outcome_type_display_name'] = df['outcome_type'].apply(lambda x: x['displayName'])
        df.drop(columns=["period", "type", "outcome_type"], inplace=True)
        
        # Reorder columns
        column_order = ['id', 'event_id', 'minute', 'second', 'team_id', 'player_id', 'x', 'y', 'end_x', 'end_y',
                        'qualifiers', 'is_touch', 'blocked_x', 'blocked_y', 'goal_mouth_z', 'goal_mouth_y', 'is_shot',
                        'card_type', 'is_goal', 'type_display_name', 'outcome_type_display_name', 'period_display_name']
        df = df[column_order]
        
        # Convert data types
        int_columns = ['id', 'event_id', 'minute', 'team_id', 'player_id']
        float_columns = ['second', 'x', 'y', 'end_x', 'end_y']
        bool_columns = ['is_shot', 'is_goal', 'card_type']
        
        df[int_columns] = df[int_columns].astype(np.int64)
        df[float_columns] = df[float_columns].astype(float)
        df[bool_columns] = df[bool_columns].fillna(False).astype(bool)
        
        # Replace NaN values in float columns with None
        for column in df.columns:
            if df[column].dtype == np.float64 or df[column].dtype == np.float32:
                df[column] = np.where(
                    np.isnan(df[column]),
                    None,
                    df[column]
                )

        #Create match_id column
        timestamp = re.split('[-: ]', match_data['timeStamp'])
        teams = [str(i) for i in df.team_id.unique()]
        match_id = int(''.join(timestamp + teams))
        df['match_id'] = match_id

        #Create player_name column
        df_names = pd.DataFrame(list(match_data['playerIdNameDictionary'].items()), columns=['PlayerId', 'PlayerName'])
        df_names['PlayerId']=df_names['PlayerId'].astype(np.int64)
        df = df.merge(df_names, left_on='player_id', right_on='PlayerId', how ='left').drop('PlayerId', axis=1)


    return match_data, match_keys, df

def insert_match_events(df, supabase, table_name):
    """
    Insert match events data into a Supabase table.

    Parameters:
    - df (pd.DataFrame): DataFrame containing match events data.
    - supabase: Supabase client instance.
    - table_name (str): Name of the Supabase table to insert data into.
    """

    class MatchEvent(BaseModel):
        """
        Pydantic model representing a match event.
        Adjust attributes based on the structure of your data.
        """
        id: int
        event_id: int
        minute: int
        second: Optional[float] = None
        team_id: int
        player_id: int
        x: float
        y: float
        end_x: Optional[float] = None
        end_y: Optional[float] = None
        qualifiers: List[dict]
        is_touch: bool
        blocked_x: Optional[float] = None
        blocked_y: Optional[float] = None
        goal_mouth_z: Optional[float] = None
        goal_mouth_y: Optional[float] = None
        is_shot: bool
        card_type: bool
        is_goal: bool
        type_display_name: str
        outcome_type_display_name: str
        period_display_name: str

    # Convert DataFrame rows to a list of dictionaries using the MatchEvent model
    events = [
        MatchEvent(**x).model_dump()
        for x in df.to_dict(orient='records')
    ]

    # Perform an upsert operation to insert or update records in the Supabase table
    execution = supabase.table(table_name).upsert(events).execute()


## Example:

In [ ]:
url = 'https://www.whoscored.com/Matches/1775627/Live/Europe-Champions-League-2023-2024-Barcelona-FC-Porto'
key = 'events'
df = get_data(url,key)

project_url = 'your-project-url'
api_key = 'your-api-key'

supabase = create_client(project_url, api_key)
table_name = 'match_events'
insert_match_events(df, supabase, table_name)

In [37]:
url = 'https://www.whoscored.com/Matches/1775627/Live/Europe-Champions-League-2023-2024-Barcelona-FC-Porto'
key = 'events'
df = get_data(url,key)

In [38]:
df

,id,event_id,minute,second,team_id,player_id,x,y,end_x,end_y,...,blocked_x,blocked_y,goal_mouth_z,goal_mouth_y,is_shot,card_type,is_goal,type_display_name,outcome_type_display_name,period_display_name
2,2622412983,3,0,0.0,297,358801,50.0,50.0,42.5,57.0,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
3,2622412999,4,0,1.0,297,351744,42.4,56.4,28.5,21.6,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
4,2622413183,5,0,8.0,297,10105,13.6,17.3,7.9,50.3,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
5,2622413231,6,0,10.0,297,373842,7.8,50.3,6.1,82.1,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
6,2622413309,7,0,14.0,297,322771,5.5,82.9,3.3,58.7,...,None,None,None,None,False,False,False,Pass,Successful,FirstHalf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2622544021,816,95,53.0,297,280060,65.5,34.9,71.6,49.9,...,None,None,None,None,False,False,False,Pass,Unsuccessful,SecondHalf
1515,2622544051,876,95,54.0,65,384711,25.5,48.1,37.2,85.0,...,None,None,None,None,False,False,False,Pass,Successful,SecondHalf
1516,2622544053,877,95,58.0,65,422938,38.8,84.3,None,None,...,None,None,None,None,False,False,False,Aerial,Successful,SecondHalf
1517,2622544069,817,95,58.0,297,409221,61.2,15.7,None,None,...,None,None,None,None,False,False,False,Aerial,Unsuccessful,SecondHalf
